In [1]:
import glob
import numpy as np
import pandas as pd
import csv
import h5py
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Layer, Dense, Dropout, LSTM, GRU, Conv1D, Conv2D, Conv2DTranspose, MaxPooling2D, AveragePooling2D, UpSampling2D
from tensorflow.keras.layers import concatenate, GlobalMaxPooling1D, Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, Reshape, TimeDistributed, Embedding, Input
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adamax, Adadelta, Adagrad, Nadam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2
from keras import backend as K
from sklearn.model_selection import KFold, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


from pycparser import parse_file as p
#from features import S0Gbg, S1Gbg, S2Gbg, S3Gbg, S4Gbg, S5Gbg, S6Gbg

from numpy.random import seed
from tensorflow.random import set_seed

from keras import backend as K

In [2]:
np.random.seed(0)
Normalise=0.005
tf.random.set_seed(0)

In [3]:
train = pd.read_csv('actualtrainprotbertnew.csv')

In [4]:
train

,Unnamed: 0,Sequence,feature
0,0,M A A A A A A A A A A G A A G G R G S G P G R ...,"[-0.3655, -0.5054, 0.557, 0.4814, -0.04416, 0...."
1,1,M A A A A A A A A A A G A A G G R G S G P G R ...,"[-0.366, -0.505, 0.557, 0.4814, -0.0437, 0.287..."
2,2,M A A A A A A A A A L G V R L R D C C S R G A ...,"[-0.539, 0.0634, 0.658, 0.597, 0.425, 0.4614, ..."
3,3,M A A A A A A A A A T E Q Q G S N G P V K K S ...,"[-0.393, 0.02625, 0.6255, 0.4604, 0.3232, 0.35..."
4,4,M A A A A A A A A A T N G T G G S S G M E V D ...,"[-0.1179, -0.6226, 0.4956, 0.3452, -0.1606, 0...."
...,...,...,...
28298,28298,M Y Y I G H P S Y Y R K H I E H V C F Q H S G ...,"[-0.5093, 0.3918, 0.2003, -0.2698, -0.1458, -0..."
28299,28299,M Y Y P V G W P K R V G L A L P G E S A S I R ...,"[-0.3154, -0.4834, 0.55, 0.5024, -0.01001, 0.2..."
28300,28300,M Y Y Q N Q H Q G K N I L S S S R M H I T S E ...,"[0.02031, -0.576, 0.3389, 0.1393, -0.0762, 0.5..."
28301,28301,M Y Y Q N Q H Q G K N I L S S S R M H I T S E ...,"[0.01884, -0.575, 0.3386, 0.1396, -0.07587, 0...."


In [5]:
X_train = train.drop(['Unnamed: 0','Sequence'],axis=1)

In [6]:
X_train

,feature
0,"[-0.3655, -0.5054, 0.557, 0.4814, -0.04416, 0...."
1,"[-0.366, -0.505, 0.557, 0.4814, -0.0437, 0.287..."
2,"[-0.539, 0.0634, 0.658, 0.597, 0.425, 0.4614, ..."
3,"[-0.393, 0.02625, 0.6255, 0.4604, 0.3232, 0.35..."
4,"[-0.1179, -0.6226, 0.4956, 0.3452, -0.1606, 0...."
...,...
28298,"[-0.5093, 0.3918, 0.2003, -0.2698, -0.1458, -0..."
28299,"[-0.3154, -0.4834, 0.55, 0.5024, -0.01001, 0.2..."
28300,"[0.02031, -0.576, 0.3389, 0.1393, -0.0762, 0.5..."
28301,"[0.01884, -0.575, 0.3386, 0.1396, -0.07587, 0...."


In [7]:
ytrain = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_train_HMM_actual.csv')

In [8]:
Y_train = ytrain[['Membrane','Cytoplasm','Nucleus','Extracellular','Cell membrane','Mitochondrion','Plastid','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [9]:
Y_train

,Membrane,Cytoplasm,Nucleus,Extracellular,Cell membrane,Mitochondrion,Plastid,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
28298,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28299,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
28300,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28301,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def parse_features(feature_string):
    feature_string = str(feature_string)
    feature_list = feature_string.strip('[]').split(',')
    feature_list = [float(val) for val in feature_list]
    return feature_list



In [11]:
X_train['feature'][0],type(X_train['feature'][0])

('[-0.3655, -0.5054, 0.557, 0.4814, -0.04416, 0.2888, -0.522, -0.1694, 0.1843, 0.08997, -0.07544, -0.4724, 0.52, -0.672, -0.08325, 0.1427, 0.1467, 0.5356, -0.11017, -0.558, -0.2216, 0.4338, 0.189, 0.463, -0.3862, -0.6084, -0.4404, 0.2773, -0.6416, 0.425, 0.3687, 0.343, 0.3035, -0.08795, 0.3772, -0.632, -0.6416, 0.1472, -0.1854, -0.6367, 0.09705, 0.524, -0.4177, -0.456, 0.02722, -0.6377, 0.4312, 0.638, -0.4011, 0.4492, 0.3748, 0.4463, -0.4375, 0.6187, 0.267, -0.089, -0.2094, 0.2109, -0.4155, 0.2988, -0.643, 0.1592, -0.596, 0.1432, 0.3862, 0.655, 0.4146, -0.6216, 0.2988, 0.256, -0.005493, 0.643, -0.2683, 0.1398, -0.6533, -0.149, 0.2659, 0.02982, 0.04935, 0.4424, -0.6777, 0.0691, 0.3875, -0.0761, 0.0906, 0.4634, -0.4883, 0.07245, 0.1274, -0.1096, -0.684, -0.2299, -0.07605, -0.526, 0.4307, 0.1685, -0.4294, -0.4805, 0.04715, 0.3376, 0.556, 0.2463, 0.02162, 0.1093, -0.1863, -0.1106, 0.10175, 0.6885, -0.1503, -0.5054, -0.3796, 0.616, 0.5776, 0.4336, -0.00255, -0.5835, -0.1768, -0.4502, -0.056

In [12]:
# Apply the parsing function to the feature column to get a list of lists
X_train['feature_list'] = X_train['feature'].apply(parse_features)# Create a new DataFrame with each feature as a separate column
X_train = pd.DataFrame(X_train['feature_list'].tolist())
X_train.columns = [f'Feat{i}' for i in range(1, 1025)]




In [13]:
X_train

,Feat1,Feat2,Feat3,Feat4,Feat5,Feat6,Feat7,Feat8,Feat9,Feat10,...,Feat1015,Feat1016,Feat1017,Feat1018,Feat1019,Feat1020,Feat1021,Feat1022,Feat1023,Feat1024
0,-0.36550,-0.50540,0.5570,0.4814,-0.04416,0.2888,-0.5220,-0.1694,0.1843,0.08997,...,-0.1129,0.2930,0.59000,0.09180,0.5550,-0.34200,0.6626,-0.18860,-0.32810,0.2890
1,-0.36600,-0.50500,0.5570,0.4814,-0.04370,0.2876,-0.5215,-0.1687,0.1840,0.08880,...,-0.1118,0.2935,0.59030,0.09064,0.5550,-0.34230,0.6620,-0.18710,-0.32960,0.2890
2,-0.53900,0.06340,0.6580,0.5970,0.42500,0.4614,-0.6226,0.0915,0.1707,-0.29830,...,0.3042,-0.2560,0.04352,-0.38750,0.5747,0.24980,-0.4045,0.09030,-0.24930,-0.1974
3,-0.39300,0.02625,0.6255,0.4604,0.32320,0.3584,-0.5270,-0.1482,0.2522,-0.26830,...,0.2842,-0.3933,-0.03207,-0.32810,0.4897,0.39260,-0.3958,0.15550,-0.12990,-0.1443
4,-0.11790,-0.62260,0.4956,0.3452,-0.16060,0.4841,-0.4820,-0.3188,0.3530,0.33640,...,-0.3523,-0.1769,0.15610,0.20000,0.4985,-0.07043,0.6104,-0.37770,0.17640,0.3225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28298,-0.50930,0.39180,0.2003,-0.2698,-0.14580,-0.2122,-0.3652,0.4260,0.4148,-0.40110,...,0.5464,-0.3113,0.34620,-0.06240,0.4438,-0.29830,-0.1919,-0.13700,-0.30420,0.3333
28299,-0.31540,-0.48340,0.5500,0.5024,-0.01001,0.2578,-0.4827,-0.1991,0.2043,0.04312,...,-0.0776,0.2708,0.56800,0.01913,0.5240,-0.27930,0.6240,-0.11237,-0.34520,0.2363
28300,0.02031,-0.57600,0.3389,0.1393,-0.07620,0.5690,-0.3420,-0.2057,0.4524,0.30900,...,-0.3206,-0.6025,-0.30440,0.08310,0.3345,0.14810,0.3777,-0.46680,0.42870,0.3464
28301,0.01884,-0.57500,0.3386,0.1396,-0.07587,0.5690,-0.3430,-0.2050,0.4526,0.30860,...,-0.3196,-0.6035,-0.30620,0.08154,0.3345,0.14970,0.3745,-0.46500,0.42920,0.3457


In [14]:
from sklearn.model_selection import train_test_split

Xtrain,Xvalidate,Ytrain,Yvalidate=train_test_split(X_train,Y_train,test_size=0.2,random_state=32)

In [15]:
Xtrain.shape,Xvalidate.shape, Ytrain.shape,Yvalidate.shape

((22642, 1024), (5661, 1024), (22642, 11), (5661, 11))

In [16]:
Xtrain.isna().sum()

Feat1       0
Feat2       0
Feat3       0
Feat4       0
Feat5       0
           ..
Feat1020    0
Feat1021    0
Feat1022    0
Feat1023    0
Feat1024    0
Length: 1024, dtype: int64

In [17]:
Xtrain.fillna(0,inplace=True)

In [18]:
Xtrain

,Feat1,Feat2,Feat3,Feat4,Feat5,Feat6,Feat7,Feat8,Feat9,Feat10,...,Feat1015,Feat1016,Feat1017,Feat1018,Feat1019,Feat1020,Feat1021,Feat1022,Feat1023,Feat1024
2404,0.01576,-0.58060,0.3083,0.1222,-0.08966,0.5527,-0.3396,-0.1884,0.44750,0.31600,...,-0.32150,-0.60000,-0.29610,0.09045,0.32000,0.1194,0.3865,-0.47310,0.4316,0.35860
26971,-0.31100,-0.48120,0.5880,0.4990,-0.04318,0.2756,-0.5044,-0.2964,0.24610,0.06647,...,-0.09310,0.18240,0.51400,0.01248,0.53370,-0.1411,0.5680,-0.03165,-0.2734,0.24770
11855,-0.36820,-0.52100,0.5460,0.4656,-0.08167,0.2769,-0.5293,-0.1769,0.19780,0.11475,...,-0.12740,0.27930,0.59300,0.11206,0.55200,-0.3508,0.6743,-0.19210,-0.3013,0.32370
14619,-0.56250,-0.44950,-0.4460,-0.0923,-0.15080,-0.2942,-0.3745,0.5874,0.00552,0.05000,...,0.09924,-0.08154,0.50440,-0.03403,-0.02783,-0.7470,0.5340,-0.35200,-0.1923,0.60800
21142,-0.12090,0.08890,0.2856,-0.2837,0.56400,0.5396,0.0691,0.5460,-0.14950,-0.54150,...,0.52200,-0.11035,0.48070,0.08260,0.13720,-0.5967,0.4827,-0.74300,-0.4814,0.43300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20414,-0.00522,-0.14390,0.4675,0.3455,0.48680,0.3152,-0.0569,-0.2510,0.24540,-0.38620,...,0.29980,-0.48200,-0.08190,-0.48560,0.09280,0.4410,-0.1893,0.17410,-0.2361,-0.08030
27516,-0.53470,-0.06183,-0.2888,-0.2229,-0.29500,-0.4490,-0.3833,0.4773,0.24170,-0.14840,...,0.34450,-0.20180,0.45580,-0.05365,0.13610,-0.5767,0.1771,-0.10810,-0.1996,0.51600
9526,-0.39360,-0.48680,0.5150,0.4668,-0.04600,0.2194,-0.5195,-0.1444,0.15400,0.05334,...,-0.06015,0.29600,0.61700,0.05804,0.51460,-0.3628,0.6426,-0.12420,-0.3462,0.32620
9771,-0.20600,0.07340,0.5340,0.3223,0.40330,0.2264,-0.2303,-0.2043,0.29100,-0.41920,...,0.39260,-0.47660,-0.04907,-0.46200,0.26220,0.4697,-0.3992,0.27120,-0.2446,-0.09980


In [153]:

    
hmm_input = Input(shape=(Xtrain.shape[1],), name='hmm_input')


d_output2 = Dense(1024, activation='sigmoid')(hmm_input)
# d_output2 = Dense(512, activation='relu')(d_output2)

d_output2 = Dense(256, activation='relu')(d_output2)
# d_output2 = Dense(128, activation='relu')(d_output2)

d_output2 = Dense(64, activation='relu')(d_output2)
d_output2 = Dense(32, activation='relu')(d_output2)
main_output = Dense(11, activation='softmax', name='main_output', kernel_regularizer=l2(0.01))(d_output2)

	

In [154]:
model = Model(inputs=[hmm_input], outputs=[main_output])
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

earlyStopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=0, mode='auto')

load_file = "78.h5"
checkpointer = ModelCheckpoint(monitor='accuracy', filepath=load_file, verbose=0, save_best_only=True)


	# history=model.fit({'bigram_input': X_biGram_Train}, {'main_output': Y_train}, epochs=500, batch_size=64, callbacks=[checkpointer, earlyStopping], verbose=0)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hmm_input (InputLayer)      [(None, 1024)]            0         
                                                                 
 dense_21 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_22 (Dense)            (None, 256)               262400    
                                                                 
 dense_23 (Dense)            (None, 64)                16448     
                                                                 
 dense_24 (Dense)            (None, 32)                2080      
                                                                 
 main_output (Dense)         (None, 11)                363       
                                                                 
Total params: 1,330,891
Trainable params: 1,330,891
Non-tra

In [155]:
with tf.device('/device:GPU:0'):
    
    history=model.fit({'hmm_input': Xtrain}, {'main_output': Ytrain},validation_data=(Xvalidate, Yvalidate), epochs=250,callbacks=[earlyStopping,checkpointer], batch_size=256)

Epoch 1/250
89/89 [==============================] - 2s 14ms/step - loss: 0.3957 - accuracy: 0.6513 - val_loss: 0.2991 - val_accuracy: 0.7205
Epoch 2/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2775 - accuracy: 0.7213 - val_loss: 0.2600 - val_accuracy: 0.7235
Epoch 3/250
89/89 [==============================] - 1s 9ms/step - loss: 0.2472 - accuracy: 0.7206 - val_loss: 0.2385 - val_accuracy: 0.7266
Epoch 4/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2292 - accuracy: 0.7234 - val_loss: 0.2245 - val_accuracy: 0.7234
Epoch 5/250
89/89 [==============================] - 1s 10ms/step - loss: 0.2187 - accuracy: 0.7204 - val_loss: 0.2152 - val_accuracy: 0.7264
Epoch 6/250
89/89 [==============================] - 1s 9ms/step - loss: 0.2120 - accuracy: 0.7210 - val_loss: 0.2118 - val_accuracy: 0.7073
Epoch 7/250
89/89 [==============================] - 1s 9ms/step - loss: 0.2068 - accuracy: 0.7195 - val_loss: 0.2057 - val_accuracy: 0.7154
Epoch 8/2

In [156]:
# from ClassLabels1 import label_generator
# from multiLabelMetrics import accuracy
# for index, row in Xtrain.iterrows():
#     single_row = Xtrain.loc[index].values.reshape(1, -1)

#     # Make predictions
#     predictions = model.predict(single_row)
#     arr_Y=np.array([Ytrain.loc[index]])
#     arr_pred = label_generator(predictions=predictions)
#     test_acc = accuracy(arr_pred,arr_Y)
    
#     x=test_acc.strictAccuracy()
#     print(x)
#     if x==100 :
#         break

In [157]:
# single_row = Xtrain.loc[index].values.reshape(1, -1)

# # Make predictions
# predictions = model.predict(single_row)
# predictions

In [158]:
# from ClassLabels1 import label_generator
# from multiLabelMetrics import accuracy
# arr_Y=np.array([Ytrain.loc[index]])
# arr_pred = label_generator(predictions=predictions)
# arr_pred

In [159]:
# arr_Y

In [160]:
# test_acc = accuracy(arr_pred,arr_Y)
# test_acc.strictAccuracy()

In [161]:
# test_acc.relaxedAccuracy()

In [162]:
model.load_weights(load_file)
score = model.evaluate({'hmm_input': Xtrain},{'main_output': Ytrain})

708/708 [==============================] - 4s 5ms/step - loss: 0.2230 - accuracy: 0.7267


In [163]:
model.load_weights(load_file)
score = model.evaluate({'hmm_input': Xvalidate},{'main_output': Yvalidate})

177/177 [==============================] - 1s 5ms/step - loss: 0.2245 - accuracy: 0.7234


In [164]:
from ClassLabels1 import label_generator
from multiLabelMetrics import accuracy
pred_scores = model.predict(Xtrain)
arr_Y = np.array(Ytrain)
arr_pred = label_generator(predictions=pred_scores)
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

708/708 [==============================] - 2s 3ms/step


53.149015104672735

In [165]:
acc.relaxedAccuracy()

91.96746191711301

In [166]:

pred_scores = model.predict(Xvalidate)
arr_Y = np.array(Yvalidate)
arr_pred = label_generator(predictions=pred_scores)

177/177 [==============================] - 1s 5ms/step


In [167]:

acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

53.27680621798269

In [168]:
acc.relaxedAccuracy()

92.00590965296847

In [169]:
import tensorflow as tf
from tensorflow.keras.models import load_model

# Specify the path to your saved model (.h5 file)
model_path = '78.h5'

# Load the model
model = load_model(model_path,compile=False)

# Print a summary of the loaded model
model.summary()

# Now you can use the loaded model for predictions or further training
correct = pd.DataFrame(columns=range(0, 12))
k=0
for i in range(len(arr_pred)):
    if np.array_equal(arr_pred[i], arr_Y[i]):
      # print(k) 
       
       correct = pd.concat([correct, pd.DataFrame([arr_Y[i]])], ignore_index=True)
       

correct.to_csv('correct1.csv')

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hmm_input (InputLayer)      [(None, 1024)]            0         
                                                                 
 dense_21 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_22 (Dense)            (None, 256)               262400    
                                                                 
 dense_23 (Dense)            (None, 64)                16448     
                                                                 
 dense_24 (Dense)            (None, 32)                2080      
                                                                 
 main_output (Dense)         (None, 11)                363       
                                                                 
Total params: 1,330,891
Trainable params: 1,330,891
Non-tra

In [170]:

test = pd.read_csv(r'C:\Users\tejus\Desktop\Final yr project\Features\Features\Actual\HMM\MatMul_test_HMM_actual.csv')

In [171]:
test

,sid,Cell membrane,Cytoplasm,Endoplasmic reticulum,Golgi apparatus,Lysosome/Vacuole,Mitochondrion,Nucleus,Peroxisome,Lengths,...,Feat_391,Feat_392,Feat_393,Feat_394,Feat_395,Feat_396,Feat_397,Feat_398,Feat_399,Feat_400
0,ENSP00000355743,0,0,0,0,0,0,1,0,496,...,0.258011,0.353851,0.435284,0.413877,0.594366,0.595805,0.396866,0.544421,0.424152,6.692077
1,ENSP00000429628,0,0,0,0,0,0,1,0,173,...,0.093635,0.185587,0.206210,0.225075,0.214002,0.339029,0.246747,0.250573,0.089538,0.442268
2,ENSP00000266544,0,0,0,0,0,1,0,0,377,...,0.278993,0.339251,0.438263,0.351402,0.544947,0.618431,0.499604,0.843471,0.315650,1.232763
3,ENSP00000470652,0,0,0,1,0,0,0,0,410,...,0.363690,0.433065,0.379793,0.382677,0.473912,0.860343,0.664466,0.945804,0.395950,0.997238
4,ENSP00000460751,0,0,0,0,0,0,1,0,163,...,0.037747,0.119094,0.092005,0.155926,0.215646,0.201815,0.094544,0.163751,0.268752,2.492428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,ENSP00000379284,0,0,0,0,0,1,0,0,372,...,0.302538,0.348803,0.334693,0.324379,0.641153,0.535000,0.460455,0.666990,0.304205,1.033432
1681,ENSP00000430164,0,1,0,0,0,0,0,0,73,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
1682,ENSP00000414655,0,1,0,0,0,0,0,0,174,...,0.014237,0.062561,0.143737,0.114862,0.048551,0.241509,0.102259,0.037878,0.008699,1.638261
1683,ENSP00000392070,0,0,0,0,0,0,1,0,327,...,0.184005,0.214807,0.337576,0.290199,0.442781,0.461941,0.351031,0.541925,0.135670,1.037373


In [172]:
Xtest=pd.read_csv('actualtestprotbert_new.csv')

In [173]:
X_test = Xtest.drop(['Unnamed: 0','fasta'],axis=1)

In [174]:
X_test['feature_list'] = X_test['feature'].apply(parse_features)# Create a new DataFrame with each feature as a separate column
X_test = pd.DataFrame(X_test['feature_list'].tolist())
X_test.columns = [f'Feat{i}' for i in range(1, 1025)]

In [175]:
X_test

,Feat1,Feat2,Feat3,Feat4,Feat5,Feat6,Feat7,Feat8,Feat9,Feat10,...,Feat1015,Feat1016,Feat1017,Feat1018,Feat1019,Feat1020,Feat1021,Feat1022,Feat1023,Feat1024
0,0.01654,-0.57800,0.3280,0.13650,-0.08550,0.5615,-0.3442,-0.19980,0.456000,0.3127,...,-0.32060,-0.60500,-0.3037,0.08170,0.33150,0.1417,0.37770,-0.4656,0.43140,0.35200
1,-0.22340,-0.57470,0.2461,0.21040,-0.20690,0.2434,-0.4438,-0.07806,0.280000,0.2670,...,-0.23560,-0.09656,0.3071,0.18190,0.38450,-0.3362,0.62060,-0.3790,0.05853,0.40920
2,-0.57400,-0.43530,-0.4414,-0.11554,-0.14160,-0.2798,-0.3755,0.59960,-0.000732,0.0404,...,0.11176,-0.09310,0.5040,-0.02777,-0.02216,-0.7500,0.52640,-0.3662,-0.19460,0.62000
3,0.11170,-0.02681,0.3657,0.13900,0.30100,0.1587,0.1619,-0.19180,0.457800,-0.4077,...,0.31520,-0.49000,-0.0865,-0.48560,0.05884,0.4302,-0.10925,0.2296,-0.28440,0.05405
4,-0.25760,-0.49930,0.2925,0.25100,-0.20850,0.1556,-0.4456,-0.09280,0.294200,0.1732,...,-0.13700,-0.01403,0.3916,0.11615,0.40970,-0.3218,0.58060,-0.2560,-0.06520,0.37350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1680,-0.47780,0.28500,0.1950,-0.16860,0.05730,-0.1155,-0.3733,0.40380,0.195000,-0.4373,...,0.56100,-0.22330,0.4360,-0.06020,0.34700,-0.3801,-0.09340,-0.2123,-0.34160,0.30860
1681,-0.36230,-0.51660,0.4941,0.41700,-0.15670,0.2191,-0.5444,-0.18270,0.238400,0.1576,...,-0.14390,0.19030,0.5330,0.11930,0.53760,-0.3171,0.62940,-0.1771,-0.21440,0.32420
1682,-0.20890,-0.61570,0.4300,0.29540,-0.22990,0.3992,-0.5060,-0.22410,0.338900,0.3398,...,-0.33060,-0.10020,0.2730,0.23520,0.50500,-0.2288,0.65100,-0.3980,0.12290,0.40260
1683,0.01212,-0.56350,0.3013,0.12415,-0.08435,0.5415,-0.3396,-0.17880,0.452100,0.2993,...,-0.30130,-0.61230,-0.3113,0.06866,0.31540,0.1368,0.34740,-0.4565,0.43070,0.34420


In [176]:
Ytest=test[['Cytoplasm','Nucleus','Cell membrane','Mitochondrion','Endoplasmic reticulum','Lysosome/Vacuole','Golgi apparatus','Peroxisome']]

In [177]:
Ytest

,Cytoplasm,Nucleus,Cell membrane,Mitochondrion,Endoplasmic reticulum,Lysosome/Vacuole,Golgi apparatus,Peroxisome
0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
1680,0,0,0,1,0,0,0,0
1681,1,0,0,0,0,0,0,0
1682,1,0,0,0,0,0,0,0
1683,0,1,0,0,0,0,0,0


In [178]:
pred_scores = model.predict(X_test)
arr_Y = np.array(Ytest)
arr_pred = label_generator(predictions=pred_scores)
print(arr_Y.shape)
print(arr_pred.shape)

53/53 [==============================] - 0s 6ms/step
(1685, 8)
(1685, 11)


In [179]:
df_arr_Y = pd.DataFrame(arr_Y)
df_arr_pred = pd.DataFrame(arr_pred)

df_arr_pred.drop(df_arr_pred.columns[[0,3,6]], axis = 1, inplace = True)

arr_Y = df_arr_Y.values
arr_pred = df_arr_pred.values
print(arr_Y.shape)
print(arr_pred.shape)

(1685, 8)
(1685, 8)


In [180]:
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy(),acc.relaxedAccuracy()

(33.76854599406528, 85.01483679525222)

In [181]:
acc = accuracy(arr_pred,arr_Y)
acc.strictAccuracy()

33.76854599406528

In [182]:
acc.relaxedAccuracy()

85.01483679525222